<a href="https://colab.research.google.com/github/nazarb/Mining/blob/main/Yolov5_test5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import os 
import zipfile

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fid = drive.ListFile({'q':"title='Wes_11_3_wgs_YOLO.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Wes_11_3_wgs_YOLO.zip')

In [3]:

import zipfile
with zipfile.ZipFile("/content/Wes_11_3_wgs_YOLO.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [4]:
import shutil
shutil.move("/content/content/Dataset", "/content/Dataset")

%rm -rf "/content/content"

In [5]:
!cat /content/Dataset/dataset.yaml

names:
- other
- mine

nc: 
- 0
- 1

path: ..
test: /content/Dataset/images/test
train: /content/Dataset/images/train
val: /content/Dataset/images/val

In [6]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "./content/Dataset/"

In [7]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15305 (delta 0), reused 1 (delta 0), pack-reused 15300
Receiving objects: 100% (15305/15305), 14.13 MiB | 7.22 MiB/s, done.
Resolving deltas: 100% (10497/10497), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 7.9 MB/s eta 0:0

In [8]:
!python train.py --img 512 --batch 16 --epochs 150 --data /content/Dataset/dataset.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/Dataset/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

In [9]:
# Validate YOLOv5s on COCO val
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/Dataset/dataset.yaml --img 512 --half

val: data=/content/Dataset/dataset.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/Dataset/labels/val.cache... 226 images, 198 backgrounds, 0 corrupt: 100% 226/226 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 8/8 [00:03<00:00,  2.02it/s]
                   all        226        121      0.632      0.438      0.495      0.219
                  mine        226        121      0.632      0.438      0.495      0.219
Speed: 0.2ms 

In [10]:
from IPython.display import Image, display


!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 512 --conf 0.5 --source /content/Dataset/images/test
display(filename='/content/Dataset/images/test/318.tif', width=600)


detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/Dataset/images/test, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/4 /content/Dataset/images/test/109.tif: 512x512 (no detections), 8.9ms
image 2/4 /content/Dataset/images/test/318.tif: 512x512 (no detections), 9.0ms
image 3/4 /content/Dataset/images/test/460.tif: 512x512 (no detections), 9.5ms
image 4/4 /content/Dataset/images/test/593.tif: 512x512 (no detections), 8.9ms
Speed:

In [11]:
!zip -r WeS_11_3_wgs_YOLO_results_1.zip /content/yolov5/runs

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/val/ (stored 0%)
  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 19%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 16%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 16%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 19%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 38%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 14%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/

In [12]:
from google.colab import drive
drive.mount('/content/drive')

import shutil

shutil.copy("/content/yolov5/WeS_11_3_wgs_YOLO_results_1.zip","/content/drive/MyDrive/Mining")

Mounted at /content/drive


'/content/drive/MyDrive/Mining/WeS_11_3_wgs_YOLO_results_1.zip'